Original

In [ ]:
# Install necessary libraries
!pip install mediapipe opencv-python

import cv2
import mediapipe as mp
import numpy as np
from IPython.display import display, Javascript, HTML
from google.colab.output import eval_js
from base64 import b64decode
from PIL import Image
import io
import threading
import time
import math

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Define eye landmark indices
LEFT_EYE_INDICES = [33, 133, 160, 159, 158, 157, 173, 153, 144, 145, 153]
RIGHT_EYE_INDICES = [362, 263, 387, 386, 385, 384, 398, 382, 381, 380, 374]

# Initialize variables for tracking
previous_center = None
speed = 0
frames = []
lock = threading.Lock()

# Function to calculate the center of the eye
def calculate_center(landmarks, indices, width, height):
    x = [landmarks[i].x for i in indices]
    y = [landmarks[i].y for i in indices]
    return (int(np.mean(x) * width), int(np.mean(y) * height))

# Function to display video frames in Colab
def display_frame(image):
    # Convert BGR to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Convert to PIL Image
    pil_img = Image.fromarray(image_rgb)
    # Display the image
    display(pil_img)

# Callback function to receive frames from JavaScript
def receive_frame(dataURL):
    global frames
    # Decode the Base64 image
    header, encoded = dataURL.split(",", 1)
    data = b64decode(encoded)
    img = Image.open(io.BytesIO(data))
    img = img.convert('RGB')
    img = np.array(img)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    with lock:
        frames.append(img)

# Register the callback
from google.colab import output
output.register_callback('notebook.receive_frame', receive_frame)

# JavaScript code to capture video frames and send to Python
def capture_video():
    display(Javascript('''
        async function startVideo() {
            const video = document.createElement('video');
            video.width = 640;
            video.height = 480;
            video.autoplay = true;
            video.style.display = 'none';
            document.body.appendChild(video);

            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            video.srcObject = stream;

            // Function to send frames to Python
            const sendFrame = () => {
                const canvas = document.createElement('canvas');
                canvas.width = video.videoWidth;
                canvas.height = video.videoHeight;
                const ctx = canvas.getContext('2d');
                ctx.drawImage(video, 0, 0, canvas.width, canvas.height);
                const dataURL = canvas.toDataURL('image/jpeg');
                google.colab.kernel.invokeFunction('notebook.receive_frame', [dataURL], {});
                setTimeout(sendFrame, 100);  // Send frame every 100ms
            }

            video.addEventListener('play', () => {
                sendFrame();
            });
        }

        startVideo();
    '''))

# Start capturing video
capture_video()

# Function to process frames and display results
def process_frames():
    global previous_center, speed
    while True:
        with lock:
            if len(frames) > 0:
                frame = frames.pop(0)
            else:
                frame = None
        if frame is not None:
            height, width, _ = frame.shape
            image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = face_mesh.process(image_rgb)
            if results.multi_face_landmarks:
                for face_landmarks in results.multi_face_landmarks:
                    # Draw face mesh
                    mp_drawing.draw_landmarks(
                        image=frame,
                        landmark_list=face_landmarks,
                        connections=mp_face_mesh.FACEMESH_TESSELATION,
                        landmark_drawing_spec=None,
                        connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style()
                    )
                    mp_drawing.draw_landmarks(
                        image=frame,
                        landmark_list=face_landmarks,
                        connections=mp_face_mesh.FACEMESH_CONTOURS,
                        landmark_drawing_spec=None,
                        connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_contours_style()
                    )

                    # Calculate centers of left and right eyes
                    left_center = calculate_center(face_landmarks.landmark, LEFT_EYE_INDICES, width, height)
                    right_center = calculate_center(face_landmarks.landmark, RIGHT_EYE_INDICES, width, height)

                    # Calculate the overall eye center
                    eye_center = ((left_center[0] + right_center[0]) // 2, (left_center[1] + right_center[1]) // 2)

                    # Draw a yellow dot at the eye center
                    cv2.circle(frame, eye_center, 5, (0, 255, 255), -1)

                    # Calculate speed based on movement
                    if previous_center is not None:
                        dx = eye_center[0] - previous_center[0]
                        dy = eye_center[1] - previous_center[1]
                        speed = math.sqrt(dx**2 + dy**2)
                    previous_center = eye_center

                    # Display speed on the frame
                    cv2.putText(frame, f'Speed: {speed:.2f}', (10, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

            # Display the processed frame
            display_frame(frame)
            # Clear previous output
            from IPython.display import clear_output
            clear_output(wait=True)
        time.sleep(0.1)  # Adjust the sleep time as needed

# Start processing frames in a separate thread
thread = threading.Thread(target=process_frames)
thread.start()

# To stop the processing, interrupt the kernel (e.g., by clicking the stop button)


In [ ]:
First

In [ ]:
# Install necessary libraries
!pip install mediapipe opencv-python ipywidgets

# Import required libraries
import cv2
import mediapipe as mp
import numpy as np
from IPython.display import display, Javascript
from google.colab import output
import base64
from PIL import Image
import io
import threading
import time
import math
import ipywidgets as widgets
from IPython.display import clear_output

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Define eye landmark indices
LEFT_EYE_INDICES = [33, 133, 160, 159, 158, 157, 173, 153, 144, 145, 153]
RIGHT_EYE_INDICES = [362, 263, 387, 386, 385, 384, 398, 382, 381, 380, 374]

# Initialize tracking variables
previous_center = None
speed = 0
prev_time = time.time()
frames = []
lock = threading.Lock()
yellow_dot_position = [112, 112]  # Starting at center of 224x224 frame

# Define helper functions
def calculate_eye_center(landmarks, indices, width, height):
    x = [landmarks[i].x for i in indices]
    y = [landmarks[i].y for i in indices]
    return (int(np.mean(x) * width), int(np.mean(y) * height))

def classify_speed(speed):
    if speed < 5:
        return "Very Slow"
    elif speed < 25:
        return "Slow"
    elif speed < 50:
        return "Normal"
    elif speed < 100:
        return "High"
    else:
        return "Very High"

# Define the callback function to receive frames from JavaScript
def receive_frame(dataURL):
    global frames
    try:
        header, encoded = dataURL.split(",", 1)
        data = base64.b64decode(encoded)
        img = Image.open(io.BytesIO(data))
        img = img.convert('RGB')
        img = np.array(img)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        with lock:
            frames.append(img)
    except Exception as e:
        print(f"Error in receive_frame: {e}")

# Register the callback
output.register_callback('notebook.receive_frame', receive_frame)

# JavaScript code to capture video frames and send to Python
def capture_video():
    display(Javascript('''
        async function startVideo() {
            const video = document.createElement('video');
            video.width = 640;
            video.height = 480;
            video.autoplay = true;
            video.style.display = 'none';
            document.body.appendChild(video);

            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            video.srcObject = stream;

            // Function to send frames to Python
            const sendFrame = () => {
                const canvas = document.createElement('canvas');
                canvas.width = video.videoWidth;
                canvas.height = video.videoHeight;
                const ctx = canvas.getContext('2d');
                ctx.drawImage(video, 0, 0, canvas.width, canvas.height);
                const dataURL = canvas.toDataURL('image/jpeg');
                google.colab.kernel.invokeFunction('notebook.receive_frame', [dataURL], {});
                setTimeout(sendFrame, 50);  // Send frame every 50ms (20 FPS)
            }

            video.addEventListener('play', () => {
                sendFrame();
            });
        }

        startVideo();
    '''))

# Start capturing video
capture_video()

# Create Output widgets for two real-time streams and info display
face_image = widgets.Image(format='jpeg', width=640, height=480)        # Face with mesh and eye movement
yellow_dot_image = widgets.Image(format='jpeg', width=224, height=224)  # Yellow dot movement
speed_label = widgets.Label(value="Saccade Speed: 0.00 pixels/sec (N/A)") # Speed and classification info

# Arrange the widgets in the notebook
hbox = widgets.HBox([face_image, yellow_dot_image])
vbox = widgets.VBox([hbox, speed_label])
display(vbox)

# Define the function to process and display frames
def process_frames():
    global previous_center, speed, prev_time, yellow_dot_position
    amplification_factor = 20  # Increased amplification for clearer movement
    while True:
        with lock:
            if len(frames) > 0:
                frame = frames.pop(0)
            else:
                frame = None
        if frame is not None:
            height, width, _ = frame.shape
            image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = face_mesh.process(image_rgb)
            yellow_dot_frame = np.zeros((224, 224, 3), dtype=np.uint8)  # Black background
            speed_info = "Saccade Speed: 0.00 pixels/sec (N/A)"
            if results.multi_face_landmarks:
                for face_landmarks in results.multi_face_landmarks:
                    # Draw face mesh on the original frame
                    mp_drawing.draw_landmarks(
                        image=frame,
                        landmark_list=face_landmarks,
                        connections=mp_face_mesh.FACEMESH_TESSELATION,
                        landmark_drawing_spec=None,
                        connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style()
                    )
                    mp_drawing.draw_landmarks(
                        image=frame,
                        landmark_list=face_landmarks,
                        connections=mp_face_mesh.FACEMESH_CONTOURS,
                        landmark_drawing_spec=None,
                        connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_contours_style()
                    )

                    # Calculate centers of left and right eyes
                    left_center = calculate_eye_center(face_landmarks.landmark, LEFT_EYE_INDICES, width, height)
                    right_center = calculate_eye_center(face_landmarks.landmark, RIGHT_EYE_INDICES, width, height)

                    # Calculate the overall eye center
                    eye_center = ((left_center[0] + right_center[0]) // 2, (left_center[1] + right_center[1]) // 2)

                    # Draw a yellow dot at the eye center in the face frame
                    cv2.circle(frame, eye_center, 15, (0, 255, 255), -1)  # Increased radius for visibility

                    # Calculate speed based on movement
                    current_time = time.time()
                    if previous_center is not None:
                        dx = eye_center[0] - previous_center[0]
                        dy = eye_center[1] - previous_center[1]
                        dt = current_time - prev_time
                        speed = math.sqrt(dx**2 + dy**2) / dt if dt > 0 else 0
                        speed_class = classify_speed(speed)
                        speed_info = f"Saccade Speed: {speed:.2f} pixels/sec ({speed_class})"
                    else:
                        speed_info = "Saccade Speed: 0.00 pixels/sec (N/A)"

                    # Update previous position and time
                    previous_center = eye_center
                    prev_time = current_time

                    # Update yellow_dot_position based on movement delta
                    yellow_dot_position[0] += int(dx * amplification_factor)
                    yellow_dot_position[1] += int(dy * amplification_factor)

                    # Clip the yellow_dot_position within the frame
                    yellow_dot_position[0] = max(0, min(223, yellow_dot_position[0]))
                    yellow_dot_position[1] = max(0, min(223, yellow_dot_position[1]))

                    # Draw the yellow dot on the separate frame
                    cv2.circle(yellow_dot_frame, (yellow_dot_position[0], yellow_dot_position[1]), 15, (0, 255, 255), -1)  # Increased radius

            # Encode the face frame as JPEG
            _, encoded_face_frame = cv2.imencode('.jpg', frame)
            face_image.value = encoded_face_frame.tobytes()

            # Encode the yellow dot frame as JPEG
            _, encoded_dot_frame = cv2.imencode('.jpg', yellow_dot_frame)
            yellow_dot_image.value = encoded_dot_frame.tobytes()

            # Update speed info
            speed_label.value = speed_info

        # Control frame rate (currently 20 FPS)
        time.sleep(0.05)

# Start processing frames in a separate thread
processing_thread = threading.Thread(target=process_frames)
processing_thread.daemon = True
processing_thread.start()

# Note:
# To stop the processing, interrupt the Colab kernel by clicking the "Stop" button.


Second

In [ ]:
# Install necessary libraries
!pip install mediapipe opencv-python ipywidgets pandas

# Import required libraries
import cv2
import mediapipe as mp
import numpy as np
from IPython.display import display, Javascript
from google.colab import output
import base64
from PIL import Image
import io
import threading
import time
import math
import ipywidgets as widgets
import pandas as pd
from datetime import datetime
import atexit

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Define eye landmark indices for left and right eyes
LEFT_EYE_INDICES = [33, 160, 158, 133, 153, 144]
RIGHT_EYE_INDICES = [362, 385, 387, 263, 373, 380]
LEFT_IRIS_INDICES = [468, 469, 470, 471, 472, 473]
RIGHT_IRIS_INDICES = [474, 475, 476, 477, 478, 479]

# Initialize tracking variables
previous_center = None
speed = 0
prev_time = time.time()
frames = []
lock = threading.Lock()
yellow_dot_position = [112, 112]  # Starting at center of 224x224 frame
blink_count = 0
blink_threshold = 0.20  # EAR threshold for blink detection (lower threshold)
blink_flag = False
fixation_start_time = None
fixation_duration = 0
fixation_threshold = 2  # pixels
data_records = []

# Define helper functions
def calculate_eye_center(landmarks, indices, width, height):
    x = [landmarks[i].x for i in indices]
    y = [landmarks[i].y for i in indices]
    return (int(np.mean(x) * width), int(np.mean(y) * height))

def calculate_pupil_diameter(landmarks, iris_indices, width, height):
    if len(iris_indices) == 0:
        return 0.0  # Return 0 if iris landmarks are not available

    iris_landmarks = [landmarks[i] for i in iris_indices if i < len(landmarks)]  # Ensure valid indices
    if len(iris_landmarks) < 2:
        return 0.0  # Return 0 if not enough landmarks for calculation

    center_x = np.mean([lm.x for lm in iris_landmarks]) * width
    center_y = np.mean([lm.y for lm in iris_landmarks]) * height
    distances = [math.sqrt((lm.x * width - center_x)**2 + (lm.y * height - center_y)**2) for lm in iris_landmarks]
    diameter = 2 * np.mean(distances)
    return diameter

def classify_speed(speed):
    if speed < 5:
        return "Very Slow"
    elif speed < 25:
        return "Slow"
    elif speed < 50:
        return "Normal"
    elif speed < 100:
        return "High"
    else:
        return "Very High"

def calculate_gaze_deviation(eye_center, width, height):
    center_x, center_y = width / 2, height / 2
    dx = eye_center[0] - center_x
    dy = eye_center[1] - center_y
    angle = math.degrees(math.atan2(dy, dx))
    return angle

def detect_blink(landmarks, width, height):
    # Eye Aspect Ratio (EAR) calculation
    def eye_aspect_ratio(eye):
        # Compute the distances between the vertical eye landmarks
        A = math.dist((eye[1].x * width, eye[1].y * height), (eye[5].x * width, eye[5].y * height))
        B = math.dist((eye[2].x * width, eye[2].y * height), (eye[4].x * width, eye[4].y * height))
        # Compute the distance between the horizontal eye landmarks
        C = math.dist((eye[0].x * width, eye[0].y * height), (eye[3].x * width, eye[3].y * height))
        # Compute EAR
        ear = (A + B) / (2.0 * C)
        return ear

    # Left eye EAR
    left_eye = [landmarks[i] for i in LEFT_EYE_INDICES if i < len(landmarks)]
    right_eye = [landmarks[i] for i in RIGHT_EYE_INDICES if i < len(landmarks)]

    if len(left_eye) == 6 and len(right_eye) == 6:
        left_ear = eye_aspect_ratio(left_eye)
        right_ear = eye_aspect_ratio(right_eye)
        avg_ear = (left_ear + right_ear) / 2.0
        return avg_ear < blink_threshold
    return False

# Define the callback function to receive frames from JavaScript
def receive_frame(dataURL):
    global frames
    try:
        header, encoded = dataURL.split(",", 1)
        data = base64.b64decode(encoded)
        img = Image.open(io.BytesIO(data))
        img = img.convert('RGB')
        img = np.array(img)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        with lock:
            frames.append(img)
    except Exception as e:
        print(f"Error in receive_frame: {e}")

# Register the callback
output.register_callback('notebook.receive_frame', receive_frame)

# JavaScript code to capture video frames and send to Python
def capture_video():
    display(Javascript('''
        async function startVideo() {
            const video = document.createElement('video');
            video.width = 640;
            video.height = 480;
            video.autoplay = true;
            video.style.display = 'none';
            document.body.appendChild(video);

            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            video.srcObject = stream;

            // Function to send frames to Python
            const sendFrame = () => {
                const canvas = document.createElement('canvas');
                canvas.width = video.videoWidth;
                canvas.height = video.videoHeight;
                const ctx = canvas.getContext('2d');
                ctx.drawImage(video, 0, 0, canvas.width, canvas.height);
                const dataURL = canvas.toDataURL('image/jpeg');
                google.colab.kernel.invokeFunction('notebook.receive_frame', [dataURL], {});
                setTimeout(sendFrame, 50);  // Send frame every 50ms (20 FPS)
            }

            video.addEventListener('play', () => {
                sendFrame();
            });
        }

        startVideo();
    '''))

# Start capturing video
capture_video()

# Create Output widgets for two real-time streams and info display
face_image = widgets.Image(format='jpeg', width=640, height=480)        # Face with mesh and eye movement
yellow_dot_image = widgets.Image(format='jpeg', width=224, height=224)  # Yellow dot movement
speed_label = widgets.Label(value="Saccade Speed: 0.00 pixels/sec (N/A)") # Speed and classification info
blink_label = widgets.Label(value="Blink Count: 0")                     # Blink count
fixation_label = widgets.Label(value="Fixation Duration: 0.00 sec")    # Fixation duration
pupil_label = widgets.Label(value="Pupil Diameter: 0.00 mm")            # Pupil diameter
gaze_label = widgets.Label(value="Gaze Deviation: 0.00 degrees")       # Gaze deviation

# Arrange the widgets in the notebook
hbox = widgets.HBox([face_image, yellow_dot_image])
vbox = widgets.VBox([hbox, speed_label, blink_label, fixation_label, pupil_label, gaze_label])
display(vbox)

# Define the function to process and display frames
def process_frames():
    global previous_center, speed, prev_time, yellow_dot_position, blink_count, blink_flag, fixation_start_time, fixation_duration, data_records
    amplification_factor = 30  # Increased amplification for clearer movement
    while True:
        with lock:
            if len(frames) > 0:
                frame = frames.pop(0)
            else:
                frame = None
        if frame is not None:
            height, width, _ = frame.shape
            image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = face_mesh.process(image_rgb)
            yellow_dot_frame = np.ones((224, 224, 3), dtype=np.uint8) * 255  # White background
            speed_info = "Saccade Speed: 0.00 pixels/sec (N/A)"
            blink_detected = False
            pupil_diameter = 0.00
            gaze_deviation = 0.00
            fixation_info = "Fixation Duration: 0.00 sec"
            dx, dy = 0, 0  # Initialize dx and dy
            speed_class = "N/A"  # Initialize speed_class

            if results.multi_face_landmarks:
                for face_landmarks in results.multi_face_landmarks:
                    # Draw face mesh on the original frame
                    mp_drawing.draw_landmarks(
                        image=frame,
                        landmark_list=face_landmarks,
                        connections=mp_face_mesh.FACEMESH_TESSELATION,
                        landmark_drawing_spec=None,
                        connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style()
                    )
                    mp_drawing.draw_landmarks(
                        image=frame,
                        landmark_list=face_landmarks,
                        connections=mp_face_mesh.FACEMESH_CONTOURS,
                        landmark_drawing_spec=None,
                        connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_contours_style()
                    )

                    # Calculate centers of left and right eyes
                    left_center = calculate_eye_center(face_landmarks.landmark, LEFT_EYE_INDICES, width, height)
                    right_center = calculate_eye_center(face_landmarks.landmark, RIGHT_EYE_INDICES, width, height)

                    # Calculate the overall eye center
                    eye_center = ((left_center[0] + right_center[0]) // 2, (left_center[1] + right_center[1]) // 2)

                    # Draw a yellow dot at the eye center in the face frame
                    cv2.circle(frame, eye_center, 15, (0, 255, 255), -1)  # Increased radius for visibility

                    # Calculate speed based on movement
                    current_time = time.time()
                    if previous_center is not None:
                        dx = eye_center[0] - previous_center[0]
                        dy = eye_center[1] - previous_center[1]
                        dt = current_time - prev_time
                        speed = math.sqrt(dx**2 + dy**2) / dt if dt > 0 else 0
                        speed_class = classify_speed(speed)
                        speed_info = f"Saccade Speed: {speed:.2f} pixels/sec ({speed_class})"
                    else:
                        speed_info = "Saccade Speed: 0.00 pixels/sec (N/A)"

                    # Update previous position and time
                    previous_center = eye_center
                    prev_time = current_time

                    # Draw the yellow dot on a separate frame (amplified)
                    rel_x = (eye_center[0] - width / 2) * amplification_factor
                    rel_y = (eye_center[1] - height / 2) * amplification_factor
                    norm_x = 112 + rel_x  # Center of 224x224 frame is 112
                    norm_y = 112 + rel_y
                    norm_x = int(np.clip(norm_x, 0, 223))
                    norm_y = int(np.clip(norm_y, 0, 223))
                    cv2.circle(yellow_dot_frame, (norm_x, norm_y), 15, (0, 255, 255), -1)  # Increased radius

                    # Pupil Diameter
                    # Using Iris landmarks for better estimation
                    pupil_diameter = calculate_pupil_diameter(face_landmarks.landmark, LEFT_IRIS_INDICES + RIGHT_IRIS_INDICES, width, height)

                    # Gaze Deviation
                    gaze_deviation = calculate_gaze_deviation(eye_center, width, height)

                    # Blink Detection
                    blink_detected = detect_blink(face_landmarks.landmark, width, height)
                    if blink_detected and not blink_flag:
                        blink_count += 1
                        blink_flag = True
                    elif not blink_detected and blink_flag:
                        blink_flag = False

                    # Fixation Duration
                    movement = math.sqrt(dx**2 + dy**2)
                    if movement < fixation_threshold:
                        if fixation_start_time is None:
                            fixation_start_time = current_time
                        else:
                            fixation_duration = current_time - fixation_start_time
                    else:
                        if fixation_start_time is not None:
                            fixation_duration = current_time - fixation_start_time
                            fixation_start_time = None

                    fixation_info = f"Fixation Duration: {fixation_duration:.2f} sec"

                    # Data Recording
                    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")
                    data_records.append({
                        "Timestamp": timestamp,
                        "Saccade_Speed_pixels_sec": speed,
                        "Saccade_Speed_Class": speed_class,
                        "Eye_Center_X": eye_center[0],
                        "Eye_Center_Y": eye_center[1],
                        "Yellow_Dot_X": norm_x,
                        "Yellow_Dot_Y": norm_y,
                        "Pupil_Diameter_mm": pupil_diameter,
                        "Gaze_Deviation_deg": gaze_deviation,
                        "Blink_Count": blink_count,
                        "Fixation_Duration_sec": fixation_duration
                    })

            # Encode the face frame as JPEG
            _, encoded_face_frame = cv2.imencode('.jpg', frame)
            face_image.value = encoded_face_frame.tobytes()

            # Encode the yellow dot frame as JPEG
            _, encoded_dot_frame = cv2.imencode('.jpg', yellow_dot_frame)
            yellow_dot_image.value = encoded_dot_frame.tobytes()

            # Update labels
            speed_label.value = speed_info
            blink_label.value = f"Blink Count: {blink_count}"
            fixation_label.value = fixation_info
            pupil_label.value = f"Pupil Diameter: {pupil_diameter:.2f} mm"
            gaze_label.value = f"Gaze Deviation: {gaze_deviation:.2f} degrees"

        # Control frame rate (20 FPS)
        time.sleep(0.05)

# Start processing frames in a separate thread
processing_thread = threading.Thread(target=process_frames)
processing_thread.daemon = True
processing_thread.start()

# Save data to CSV when the kernel is interrupted
def save_data():
    global data_records
    if data_records:
        df = pd.DataFrame(data_records)
        filename = f"eye_movement_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        df.to_csv(filename, index=False)
        print(f"Data saved to {filename}")
    else:
        print("No data to save.")

atexit.register(save_data)


Third

In [ ]:
# Install necessary libraries
!pip install mediapipe opencv-python ipywidgets pandas

# Import required libraries
import cv2
import mediapipe as mp
import numpy as np
from IPython.display import display, Javascript
from google.colab import output
import base64
from PIL import Image
import io
import threading
import time
import math
import ipywidgets as widgets
import pandas as pd
from datetime import datetime
import atexit

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.7,  # Increased confidence for better accuracy
    min_tracking_confidence=0.7
)

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Define eye landmark indices for left and right eyes
LEFT_EYE_INDICES = [33, 160, 158, 133, 153, 144]
RIGHT_EYE_INDICES = [362, 385, 387, 263, 373, 380]
LEFT_IRIS_INDICES = [468, 469, 470, 471, 472, 473]
RIGHT_IRIS_INDICES = [474, 475, 476, 477, 478, 479]

# Initialize tracking variables
previous_center = None
speed = 0
prev_time = time.time()
frames = []
lock = threading.Lock()
yellow_dot_position = [112, 112]  # Starting at center of 224x224 frame
blink_count = 0
blink_threshold = 0.20  # EAR threshold for blink detection (lower threshold)
blink_flag = False
fixation_start_time = None
fixation_duration = 0
fixation_threshold = 2  # pixels
data_records = []
is_streaming = True  # Flag to control video stream

# Define helper functions
def calculate_eye_center(landmarks, indices, width, height):
    x = [landmarks[i].x for i in indices]
    y = [landmarks[i].y for i in indices]
    return (int(np.mean(x) * width), int(np.mean(y) * height))

def calculate_pupil_diameter(landmarks, iris_indices, width, height):
    if len(iris_indices) == 0:
        return 0.0  # Return 0 if iris landmarks are not available

    iris_landmarks = [landmarks[i] for i in iris_indices if i < len(landmarks)]  # Ensure valid indices
    if len(iris_landmarks) < 2:
        return 0.0  # Return 0 if not enough landmarks for calculation

    center_x = np.mean([lm.x for lm in iris_landmarks]) * width
    center_y = np.mean([lm.y for lm in iris_landmarks]) * height
    distances = [math.sqrt((lm.x * width - center_x)**2 + (lm.y * height - center_y)**2) for lm in iris_landmarks]
    diameter = 2 * np.mean(distances)
    return diameter

def classify_speed(speed):
    if speed < 5:
        return "Very Slow"
    elif speed < 25:
        return "Slow"
    elif speed < 50:
        return "Normal"
    elif speed < 100:
        return "High"
    else:
        return "Very High"

def calculate_gaze_deviation(eye_center, width, height):
    center_x, center_y = width / 2, height / 2
    dx = eye_center[0] - center_x
    dy = eye_center[1] - center_y
    angle = math.degrees(math.atan2(dy, dx))
    return angle

def detect_blink(landmarks, width, height):
    # Eye Aspect Ratio (EAR) calculation
    def eye_aspect_ratio(eye):
        # Compute the distances between the vertical eye landmarks
        A = math.dist((eye[1].x * width, eye[1].y * height), (eye[5].x * width, eye[5].y * height))
        B = math.dist((eye[2].x * width, eye[2].y * height), (eye[4].x * width, eye[4].y * height))
        # Compute the distance between the horizontal eye landmarks
        C = math.dist((eye[0].x * width, eye[0].y * height), (eye[3].x * width, eye[3].y * height))
        # Compute EAR
        ear = (A + B) / (2.0 * C)
        return ear

    # Left eye EAR
    left_eye = [landmarks[i] for i in LEFT_EYE_INDICES if i < len(landmarks)]
    right_eye = [landmarks[i] for i in RIGHT_EYE_INDICES if i < len(landmarks)]

    if len(left_eye) == 6 and len(right_eye) == 6:
        left_ear = eye_aspect_ratio(left_eye)
        right_ear = eye_aspect_ratio(right_eye)
        avg_ear = (left_ear + right_ear) / 2.0
        return avg_ear < blink_threshold
    return False

# Define the callback function to receive frames from JavaScript
def receive_frame(dataURL):
    global frames
    try:
        header, encoded = dataURL.split(",", 1)
        data = base64.b64decode(encoded)
        img = Image.open(io.BytesIO(data))
        img = img.convert('RGB')
        img = np.array(img)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        with lock:
            frames.append(img)
    except Exception as e:
        print(f"Error in receive_frame: {e}")

# Register the callback
output.register_callback('notebook.receive_frame', receive_frame)

# JavaScript code to capture video frames and send to Python
def capture_video():
    display(Javascript('''
        async function startVideo() {
            const video = document.createElement('video');
            video.width = 640;
            video.height = 480;
            video.autoplay = true;
            video.style.display = 'none';
            document.body.appendChild(video);

            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            video.srcObject = stream;

            // Function to send frames to Python
            const sendFrame = () => {
                const canvas = document.createElement('canvas');
                canvas.width = video.videoWidth;
                canvas.height = video.videoHeight;
                const ctx = canvas.getContext('2d');
                ctx.drawImage(video, 0, 0, canvas.width, canvas.height);
                const dataURL = canvas.toDataURL('image/jpeg');
                google.colab.kernel.invokeFunction('notebook.receive_frame', [dataURL], {});
                setTimeout(sendFrame, 50);  // Send frame every 50ms (20 FPS)
            }

            video.addEventListener('play', () => {
                sendFrame();
            });
        }

        startVideo();
    '''))

# Start capturing video
capture_video()

# Create Output widgets for two real-time streams and info display
face_image = widgets.Image(format='jpeg', width=640, height=480)        # Face with mesh and eye movement
yellow_dot_image = widgets.Image(format='jpeg', width=224, height=224)  # Yellow dot movement
speed_label = widgets.Label(value="Saccade Speed: 0.00 pixels/sec (N/A)") # Speed and classification info
blink_label = widgets.Label(value="Blink Count: 0")                     # Blink count
fixation_label = widgets.Label(value="Fixation Duration: 0.00 sec")    # Fixation duration
pupil_label = widgets.Label(value="Pupil Diameter: 0.00 mm")            # Pupil diameter
gaze_label = widgets.Label(value="Gaze Deviation: 0.00 degrees")       # Gaze deviation

# Button to stop the video stream
stop_button = widgets.Button(description="Stop Stream", button_style='danger')

# Arrange the widgets in the notebook
hbox = widgets.HBox([face_image, yellow_dot_image])
vbox = widgets.VBox([hbox, speed_label, blink_label, fixation_label, pupil_label, gaze_label, stop_button])
display(vbox)

# Define the function to process and display frames
def process_frames():
    global previous_center, speed, prev_time, yellow_dot_position, blink_count, blink_flag, fixation_start_time, fixation_duration, data_records, is_streaming
    amplification_factor = 30  # Increased amplification for clearer movement
    while is_streaming:
        with lock:
            if len(frames) > 0:
                frame = frames.pop(0)
            else:
                frame = None
        if frame is not None:
            height, width, _ = frame.shape
            image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = face_mesh.process(image_rgb)
            yellow_dot_frame = np.ones((224, 224, 3), dtype=np.uint8) * 255  # White background
            speed_info = "Saccade Speed: 0.00 pixels/sec (N/A)"
            blink_detected = False
            pupil_diameter = 0.00
            gaze_deviation = 0.00
            fixation_info = "Fixation Duration: 0.00 sec"
            dx, dy = 0, 0  # Initialize dx and dy
            speed_class = "N/A"  # Initialize speed_class

            if results.multi_face_landmarks:
                for face_landmarks in results.multi_face_landmarks:
                    # Draw face mesh on the original frame
                    mp_drawing.draw_landmarks(
                        image=frame,
                        landmark_list=face_landmarks,
                        connections=mp_face_mesh.FACEMESH_TESSELATION,
                        landmark_drawing_spec=None,
                        connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style()
                    )
                    mp_drawing.draw_landmarks(
                        image=frame,
                        landmark_list=face_landmarks,
                        connections=mp_face_mesh.FACEMESH_CONTOURS,
                        landmark_drawing_spec=None,
                        connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_contours_style()
                    )

                    # Calculate centers of left and right eyes
                    left_center = calculate_eye_center(face_landmarks.landmark, LEFT_EYE_INDICES, width, height)
                    right_center = calculate_eye_center(face_landmarks.landmark, RIGHT_EYE_INDICES, width, height)

                    # Calculate the overall eye center
                    eye_center = ((left_center[0] + right_center[0]) // 2, (left_center[1] + right_center[1]) // 2)

                    # Draw a yellow dot at the eye center in the face frame
                    cv2.circle(frame, eye_center, 15, (0, 255, 255), -1)  # Increased radius for visibility

                    # Calculate speed based on movement
                    current_time = time.time()
                    if previous_center is not None:
                        dx = eye_center[0] - previous_center[0]
                        dy = eye_center[1] - previous_center[1]
                        dt = current_time - prev_time
                        speed = math.sqrt(dx**2 + dy**2) / dt if dt > 0 else 0
                        speed_class = classify_speed(speed)
                        speed_info = f"Saccade Speed: {speed:.2f} pixels/sec ({speed_class})"
                    else:
                        speed_info = "Saccade Speed: 0.00 pixels/sec (N/A)"

                    # Update previous position and time
                    previous_center = eye_center
                    prev_time = current_time

                    # Draw the yellow dot on a separate frame (amplified)
                    rel_x = (eye_center[0] - width / 2) * amplification_factor
                    rel_y = (eye_center[1] - height / 2) * amplification_factor
                    norm_x = 112 + rel_x  # Center of 224x224 frame is 112
                    norm_y = 112 + rel_y
                    norm_x = int(np.clip(norm_x, 0, 223))
                    norm_y = int(np.clip(norm_y, 0, 223))
                    cv2.circle(yellow_dot_frame, (norm_x, norm_y), 15, (0, 255, 255), -1)  # Increased radius

                    # Pupil Diameter
                    # Using Iris landmarks for better estimation
                    pupil_diameter = calculate_pupil_diameter(face_landmarks.landmark, LEFT_IRIS_INDICES + RIGHT_IRIS_INDICES, width, height)

                    # Gaze Deviation
                    gaze_deviation = calculate_gaze_deviation(eye_center, width, height)

                    # Blink Detection
                    blink_detected = detect_blink(face_landmarks.landmark, width, height)
                    if blink_detected and not blink_flag:
                        blink_count += 1
                        blink_flag = True
                    elif not blink_detected and blink_flag:
                        blink_flag = False

                    # Fixation Duration
                    movement = math.sqrt(dx**2 + dy**2)
                    if movement < fixation_threshold:
                        if fixation_start_time is None:
                            fixation_start_time = current_time
                        else:
                            fixation_duration = current_time - fixation_start_time
                    else:
                        if fixation_start_time is not None:
                            fixation_duration = current_time - fixation_start_time
                            fixation_start_time = None

                    fixation_info = f"Fixation Duration: {fixation_duration:.2f} sec"

                    # Data Recording
                    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")
                    data_records.append({
                        "Timestamp": timestamp,
                        "Saccade_Speed_pixels_sec": speed,
                        "Saccade_Speed_Class": speed_class,
                        "Eye_Center_X": eye_center[0],
                        "Eye_Center_Y": eye_center[1],
                        "Yellow_Dot_X": norm_x,
                        "Yellow_Dot_Y": norm_y,
                        "Pupil_Diameter_mm": pupil_diameter,
                        "Gaze_Deviation_deg": gaze_deviation,
                        "Blink_Count": blink_count,
                        "Fixation_Duration_sec": fixation_duration
                    })

            # Encode the face frame as JPEG
            _, encoded_face_frame = cv2.imencode('.jpg', frame)
            face_image.value = encoded_face_frame.tobytes()

            # Encode the yellow dot frame as JPEG
            _, encoded_dot_frame = cv2.imencode('.jpg', yellow_dot_frame)
            yellow_dot_image.value = encoded_dot_frame.tobytes()

            # Update labels
            speed_label.value = speed_info
            blink_label.value = f"Blink Count: {blink_count}"
            fixation_label.value = fixation_info
            pupil_label.value = f"Pupil Diameter: {pupil_diameter:.2f} mm"
            gaze_label.value = f"Gaze Deviation: {gaze_deviation:.2f} degrees"

        # Control frame rate (20 FPS)
        time.sleep(0.05)

# Button event handler to stop the stream
def stop_stream(b):
    global is_streaming
    is_streaming = False
    save_data()

# Attach the event handler to the button
stop_button.on_click(stop_stream)

# Start processing frames in a separate thread
processing_thread = threading.Thread(target=process_frames)
processing_thread.daemon = True
processing_thread.start()

# Save data to CSV when the stop button is clicked
def save_data():
    global data_records
    if data_records:
        df = pd.DataFrame(data_records)
        filename = f"eye_movement_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        df.to_csv(filename, index=False)
        print(f"Data saved to {filename}")
    else:
        print("No data to save.")

# Automatically save data when the kernel is interrupted
atexit.register(save_data)


Forth

In [ ]:
# Install necessary libraries
!pip install mediapipe opencv-python ipywidgets pandas

# Import required libraries
import cv2
import mediapipe as mp
import numpy as np
from IPython.display import display, Javascript
from google.colab import output
import base64
from PIL import Image
import io
import threading
import time
import math
import ipywidgets as widgets
import pandas as pd
from datetime import datetime
import atexit

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.7,  # Increased confidence for better accuracy
    min_tracking_confidence=0.7
)

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Define eye landmark indices for left and right eyes
LEFT_EYE_INDICES = [33, 160, 158, 133, 153, 144]
RIGHT_EYE_INDICES = [362, 385, 387, 263, 373, 380]
LEFT_IRIS_INDICES = [468, 469, 470, 471, 472, 473]
RIGHT_IRIS_INDICES = [474, 475, 476, 477, 478, 479]

# Initialize tracking variables
previous_center = None
speed = 0
prev_time = time.time()
frames = []
lock = threading.Lock()
yellow_dot_position = [112, 112]  # Starting at center of 224x224 frame
blink_count = 0
blink_threshold = 0.20  # EAR threshold for blink detection (lower threshold)
blink_flag = False
fixation_start_time = None
fixation_duration = 0
fixation_threshold = 2  # pixels
data_records = []
is_streaming = True  # Flag to control video stream
saccade_start_time = None
saccadic_latency = 0
smooth_pursuit_gain = 0
search_time_start = None
total_search_time = 0
target_search_count = 0

# Define helper functions
def calculate_eye_center(landmarks, indices, width, height):
    x = [landmarks[i].x for i in indices]
    y = [landmarks[i].y for i in indices]
    return (int(np.mean(x) * width), int(np.mean(y) * height))

def calculate_pupil_diameter(landmarks, iris_indices, width, height):
    if len(iris_indices) == 0:
        return 0.0  # Return 0 if iris landmarks are not available

    iris_landmarks = [landmarks[i] for i in iris_indices if i < len(landmarks)]  # Ensure valid indices
    if len(iris_landmarks) < 2:
        return 0.0  # Return 0 if not enough landmarks for calculation

    center_x = np.mean([lm.x for lm in iris_landmarks]) * width
    center_y = np.mean([lm.y for lm in iris_landmarks]) * height
    distances = [math.sqrt((lm.x * width - center_x)**2 + (lm.y * height - center_y)**2) for lm in iris_landmarks]
    diameter = 2 * np.mean(distances)
    return diameter

def classify_speed(speed):
    if speed < 5:
        return "Very Slow"
    elif speed < 25:
        return "Slow"
    elif speed < 50:
        return "Normal"
    elif speed < 100:
        return "High"
    else:
        return "Very High"

def calculate_gaze_deviation(eye_center, width, height):
    center_x, center_y = width / 2, height / 2
    dx = eye_center[0] - center_x
    dy = eye_center[1] - center_y
    angle = math.degrees(math.atan2(dy, dx))
    return angle

def detect_blink(landmarks, width, height):
    # Eye Aspect Ratio (EAR) calculation
    def eye_aspect_ratio(eye):
        # Compute the distances between the vertical eye landmarks
        A = math.dist((eye[1].x * width, eye[1].y * height), (eye[5].x * width, eye[5].y * height))
        B = math.dist((eye[2].x * width, eye[2].y * height), (eye[4].x * width, eye[4].y * height))
        # Compute the distance between the horizontal eye landmarks
        C = math.dist((eye[0].x * width, eye[0].y * height), (eye[3].x * width, eye[3].y * height))
        # Compute EAR
        ear = (A + B) / (2.0 * C)
        return ear

    # Left eye EAR
    left_eye = [landmarks[i] for i in LEFT_EYE_INDICES if i < len(landmarks)]
    right_eye = [landmarks[i] for i in RIGHT_EYE_INDICES if i < len(landmarks)]

    if len(left_eye) == 6 and len(right_eye) == 6:
        left_ear = eye_aspect_ratio(left_eye)
        right_ear = eye_aspect_ratio(right_eye)
        avg_ear = (left_ear + right_ear) / 2.0
        return avg_ear < blink_threshold
    return False

def calculate_smooth_pursuit_gain(eye_center, previous_center, width, height):
    if previous_center is None:
        return 0.0

    # Calculate movement distance
    eye_movement = math.sqrt((eye_center[0] - previous_center[0])**2 + (eye_center[1] - previous_center[1])**2)
    # Approximate target velocity using movement (can be improved with actual target data)
    target_movement = math.sqrt(width**2 + height**2) / 60  # Assuming a screen-sized target moving at 60Hz
    return eye_movement / target_movement

# Define the callback function to receive frames from JavaScript
def receive_frame(dataURL):
    global frames
    try:
        header, encoded = dataURL.split(",", 1)
        data = base64.b64decode(encoded)
        img = Image.open(io.BytesIO(data))
        img = img.convert('RGB')
        img = np.array(img)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        with lock:
            frames.append(img)
    except Exception as e:
        print(f"Error in receive_frame: {e}")

# Register the callback
output.register_callback('notebook.receive_frame', receive_frame)

# JavaScript code to capture video frames and send to Python
def capture_video():
    display(Javascript('''
        async function startVideo() {
            const video = document.createElement('video');
            video.width = 640;
            video.height = 480;
            video.autoplay = true;
            video.style.display = 'none';
            document.body.appendChild(video);

            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            video.srcObject = stream;

            // Function to send frames to Python
            const sendFrame = () => {
                const canvas = document.createElement('canvas');
                canvas.width = video.videoWidth;
                canvas.height = video.videoHeight;
                const ctx = canvas.getContext('2d');
                ctx.drawImage(video, 0, 0, canvas.width, canvas.height);
                const dataURL = canvas.toDataURL('image/jpeg');
                google.colab.kernel.invokeFunction('notebook.receive_frame', [dataURL], {});
                setTimeout(sendFrame, 50);  // Send frame every 50ms (20 FPS)
            }

            video.addEventListener('play', () => {
                sendFrame();
            });
        }

        startVideo();
    '''))

# Start capturing video
capture_video()

# Create Output widgets for two real-time streams and info display
face_image = widgets.Image(format='jpeg', width=640, height=480)        # Face with mesh and eye movement
yellow_dot_image = widgets.Image(format='jpeg', width=224, height=224)  # Yellow dot movement
speed_label = widgets.Label(value="Saccade Speed: 0.00 pixels/sec (N/A)") # Speed and classification info
blink_label = widgets.Label(value="Blink Count: 0")                     # Blink count
fixation_label = widgets.Label(value="Fixation Duration: 0.00 sec")    # Fixation duration
pupil_label = widgets.Label(value="Pupil Diameter: 0.00 mm")            # Pupil diameter
gaze_label = widgets.Label(value="Gaze Deviation: 0.00 degrees")       # Gaze deviation
latency_label = widgets.Label(value="Saccadic Latency: 0 ms")           # Saccadic latency
smooth_pursuit_label = widgets.Label(value="Smooth Pursuit Gain: 0.00") # Smooth pursuit gain
search_time_label = widgets.Label(value="Search Time: 0 ms")            # Search time

# Button to stop the video stream
stop_button = widgets.Button(description="Stop Stream", button_style='danger')

# Arrange the widgets in the notebook
hbox = widgets.HBox([face_image, yellow_dot_image])
vbox = widgets.VBox([hbox, speed_label, blink_label, fixation_label, pupil_label, gaze_label, latency_label, smooth_pursuit_label, search_time_label, stop_button])
display(vbox)

# Define the function to process and display frames
def process_frames():
    global previous_center, speed, prev_time, yellow_dot_position, blink_count, blink_flag, fixation_start_time, fixation_duration, data_records, is_streaming
    global saccade_start_time, saccadic_latency, smooth_pursuit_gain, search_time_start, total_search_time, target_search_count
    amplification_factor = 30  # Increased amplification for clearer movement
    while is_streaming:
        with lock:
            if len(frames) > 0:
                frame = frames.pop(0)
            else:
                frame = None
        if frame is not None:
            height, width, _ = frame.shape
            image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = face_mesh.process(image_rgb)
            yellow_dot_frame = np.ones((224, 224, 3), dtype=np.uint8) * 255  # White background
            speed_info = "Saccade Speed: 0.00 pixels/sec (N/A)"
            blink_detected = False
            pupil_diameter = 0.00
            gaze_deviation = 0.00
            fixation_info = "Fixation Duration: 0.00 sec"
            dx, dy = 0, 0  # Initialize dx and dy
            speed_class = "N/A"  # Initialize speed_class

            if results.multi_face_landmarks:
                for face_landmarks in results.multi_face_landmarks:
                    # Draw face mesh on the original frame
                    mp_drawing.draw_landmarks(
                        image=frame,
                        landmark_list=face_landmarks,
                        connections=mp_face_mesh.FACEMESH_TESSELATION,
                        landmark_drawing_spec=None,
                        connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style()
                    )
                    mp_drawing.draw_landmarks(
                        image=frame,
                        landmark_list=face_landmarks,
                        connections=mp_face_mesh.FACEMESH_CONTOURS,
                        landmark_drawing_spec=None,
                        connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_contours_style()
                    )

                    # Calculate centers of left and right eyes
                    left_center = calculate_eye_center(face_landmarks.landmark, LEFT_EYE_INDICES, width, height)
                    right_center = calculate_eye_center(face_landmarks.landmark, RIGHT_EYE_INDICES, width, height)

                    # Calculate the overall eye center
                    eye_center = ((left_center[0] + right_center[0]) // 2, (left_center[1] + right_center[1]) // 2)

                    # Draw a yellow dot at the eye center in the face frame
                    cv2.circle(frame, eye_center, 15, (0, 255, 255), -1)  # Increased radius for visibility

                    # Calculate speed based on movement
                    current_time = time.time()
                    if previous_center is not None:
                        dx = eye_center[0] - previous_center[0]
                        dy = eye_center[1] - previous_center[1]
                        dt = current_time - prev_time
                        speed = math.sqrt(dx**2 + dy**2) / dt if dt > 0 else 0
                        speed_class = classify_speed(speed)
                        speed_info = f"Saccade Speed: {speed:.2f} pixels/sec ({speed_class})"
                    else:
                        speed_info = "Saccade Speed: 0.00 pixels/sec (N/A)"

                    # Update previous position and time
                    previous_center = eye_center
                    prev_time = current_time

                    # Calculate Smooth Pursuit Gain
                    smooth_pursuit_gain = calculate_smooth_pursuit_gain(eye_center, previous_center, width, height)

                    # Draw the yellow dot on a separate frame (amplified)
                    rel_x = (eye_center[0] - width / 2) * amplification_factor
                    rel_y = (eye_center[1] - height / 2) * amplification_factor
                    norm_x = 112 + rel_x  # Center of 224x224 frame is 112
                    norm_y = 112 + rel_y
                    norm_x = int(np.clip(norm_x, 0, 223))
                    norm_y = int(np.clip(norm_y, 0, 223))
                    cv2.circle(yellow_dot_frame, (norm_x, norm_y), 15, (0, 255, 255), -1)  # Increased radius

                    # Pupil Diameter
                    # Using Iris landmarks for better estimation
                    pupil_diameter = calculate_pupil_diameter(face_landmarks.landmark, LEFT_IRIS_INDICES + RIGHT_IRIS_INDICES, width, height)

                    # Gaze Deviation
                    gaze_deviation = calculate_gaze_deviation(eye_center, width, height)

                    # Blink Detection
                    blink_detected = detect_blink(face_landmarks.landmark, width, height)
                    if blink_detected and not blink_flag:
                        blink_count += 1
                        blink_flag = True
                    elif not blink_detected and blink_flag:
                        blink_flag = False

                    # Saccadic Latency
                    if saccade_start_time is None:
                        saccade_start_time = current_time
                    else:
                        saccadic_latency = (current_time - saccade_start_time) * 1000  # in ms
                        saccade_start_time = None

                    # Fixation Duration
                    movement = math.sqrt(dx**2 + dy**2)
                    if movement < fixation_threshold:
                        if fixation_start_time is None:
                            fixation_start_time = current_time
                        else:
                            fixation_duration = current_time - fixation_start_time
                    else:
                        if fixation_start_time is not None:
                            fixation_duration = current_time - fixation_start_time
                            fixation_start_time = None

                    fixation_info = f"Fixation Duration: {fixation_duration:.2f} sec"

                    # Search Time in Serial Search
                    if search_time_start is None:
                        search_time_start = current_time
                    else:
                        total_search_time = (current_time - search_time_start) * 1000  # in ms

                    # Data Recording
                    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")
                    data_records.append({
                        "Timestamp": timestamp,
                        "Saccade_Speed_pixels_sec": speed,
                        "Saccade_Speed_Class": speed_class,
                        "Eye_Center_X": eye_center[0],
                        "Eye_Center_Y": eye_center[1],
                        "Yellow_Dot_X": norm_x,
                        "Yellow_Dot_Y": norm_y,
                        "Pupil_Diameter_mm": pupil_diameter,
                        "Gaze_Deviation_deg": gaze_deviation,
                        "Blink_Count": blink_count,
                        "Fixation_Duration_sec": fixation_duration,
                        "Saccadic_Latency_ms": saccadic_latency,
                        "Smooth_Pursuit_Gain": smooth_pursuit_gain,
                        "Search_Time_ms": total_search_time
                    })

            # Encode the face frame as JPEG
            _, encoded_face_frame = cv2.imencode('.jpg', frame)
            face_image.value = encoded_face_frame.tobytes()

            # Encode the yellow dot frame as JPEG
            _, encoded_dot_frame = cv2.imencode('.jpg', yellow_dot_frame)
            yellow_dot_image.value = encoded_dot_frame.tobytes()

            # Update labels
            speed_label.value = speed_info
            blink_label.value = f"Blink Count: {blink_count}"
            fixation_label.value = fixation_info
            pupil_label.value = f"Pupil Diameter: {pupil_diameter:.2f} mm"
            gaze_label.value = f"Gaze Deviation: {gaze_deviation:.2f} degrees"
            latency_label.value = f"Saccadic Latency: {saccadic_latency:.2f} ms"
            smooth_pursuit_label.value = f"Smooth Pursuit Gain: {smooth_pursuit_gain:.2f}"
            search_time_label.value = f"Search Time: {total_search_time:.2f} ms"

        # Control frame rate (20 FPS)
        time.sleep(0.05)

# Button event handler to stop the stream
def stop_stream(b):
    global is_streaming
    is_streaming = False
    save_data()

# Attach the event handler to the button
stop_button.on_click(stop_stream)

# Start processing frames in a separate thread
processing_thread = threading.Thread(target=process_frames)
processing_thread.daemon = True
processing_thread.start()

# Save data to CSV when the stop button is clicked
def save_data():
    global data_records
    if data_records:
        df = pd.DataFrame(data_records)
        filename = f"eye_movement_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        df.to_csv(filename, index=False)
        print(f"Data saved to {filename}")
    else:
        print("No data to save.")

# Automatically save data when the kernel is interrupted
atexit.register(save_data)
